In [157]:
import datetime as dt
import wget
import os
import pandas as pd
import numpy as np
from zipfile import ZipFile
from timeit import default_timer as timer
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField

## Constantes

In [4]:
# define o ticker do ativo que vou usar pra treinar o modelo
ticker = "BTCUSDT"
# timeframes precisa estar em ordem crescente e começar em 1s ou 1m
timeframes = ("1m", "5m", "15m", "30m", "1h", "2h", "4h", "8h", "1d")
# quantos períodos vamos olhar pro passado
lookback = 20

In [3]:
# só extrai o arquivo dado como parâmetro para a pasta ./Dados/temp
def extrai_arquivo(arq):
    try:
        ZipFile(arq, 'r').extractall('./Dados/temp/')
    except:
        print(f"erro ao extrair {arq}")
# mazalves@inf.ufpr.br ou marco.eu@gmail.com

In [4]:
# Adiciona e preenche linhas faltantes nos arquivos com 0
def corrige_arquivos(timeframes):
    freq = [item.replace("m", "T") for item in timeframes]
    for timeframe, freqs in zip(timeframes, freq):
        df = (pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", index_col ="Close time"))
        df.index = pd.to_datetime(df.index, unit="ms")
        novo_index= pd.date_range(start = df.index[0], end=df.index[-1], freq=freqs)
        df = df.reindex(novo_index, fill_value=0)
        # faz virar unix de volta
        df.index = (df.index - pd.Timestamp("1970-01-01")) // pd.Timedelta("1ms")
        df.reset_index(inplace=True)
        df.rename(columns={"index": "Close time"}, inplace=True)
        df.to_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", index=False)

In [5]:
# baixa dados, concatena em um dataframe só e salva em um .csv
# limpa depois
def baixa_e_concatena(ticker, timeframe, ano_inicial):
    ano_corrente, mes_corrente, dia_corrente = [dt.date.today().year, dt.date.today().month, dt.date.today().day]
    
    # baixa dados da binance conforme ticker e timeframe selecionados para a pasta ./Dados/
    # timeframes disponiveis: 12h 15m 1d 1h 1m 1mo 1s 1w 2h 30m 3d 3m 4h 5m 6h 8h
    # tickers disponiveis: https://data.binance.vision/?prefix=data/spot/monthly/klines/
    url = "https://data.binance.vision/data/spot/monthly/klines/"
    if not os.path.exists(f"./Dados/Processados/{ticker}-{timeframe}.csv"):
        for ano in range(ano_inicial, ano_corrente+1):
            for mes in range(1,12+1):
                mes = str(mes).zfill(2)
                if not ((os.path.exists(f"./Dados/temp/{ticker}-{timeframe}-{ano}-{mes}.zip"))):
                    try:
                        wget.download(f"{url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip"
                                      , out = f"./Dados/temp/")
                        pass
                    except:
                        print(f"\nFalha ao baixar {url}{ticker}/{timeframe}/{ticker}-{timeframe}-{ano}-{mes}.zip")
                else:
                    print(f"{ano}/{mes} já baixado")
    else:
        print(f"{ticker}-{timeframe} já processado")
        return
    
    # cria uma lista de arquivos do ticker e timeframe selecionado
    lista_arquivos = os.listdir("./Dados/temp/")
    lista_arquivos = [x for x in lista_arquivos if x.startswith(f"{ticker}-{timeframe}")]
    lista_arquivos[-5:]
    
    # cria um dataframe vazio pra colocar todos os dados dentro
    nomes = ["Open time","Open","High","Low","Close","Volume","Close time","Quote asset volume"
                                 ,"Number of trades","Taker buy base asset volume","Taker buy quote asset volume","Ignore"]
    df = pd.DataFrame(columns = nomes)
    
    # concatena tudo em um CSV e deixa na pasta ./Dados/Processados/
    for arq in lista_arquivos:
        extrai_arquivo(f"./Dados/temp/{arq}")
        df = pd.concat([df, pd.read_csv(f'./Dados/temp/{arq[:-4]}.csv', sep=',',decimal='.'
                                   , encoding='latin1', names=nomes, header=None)], ignore_index=True, copy=False)
        os.remove(f"./Dados/temp/{arq[:-4]}.csv")
    df.drop("Ignore", inplace=True, axis=1)
    df.set_index("Open time", inplace=True)
    df.to_csv(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    print(f"./Dados/Processados/{ticker}-{timeframe}.csv")
    
    # deleta tudo que é temporario e já foi processado
    for arq in lista_arquivos:
        os.remove(f"./Dados/temp/{arq}")
    
    
    return

## Prepara os dados

#### Só rodar se precisar baixar e processar tudo de novo!

In [6]:
for timeframe in timeframes:
    baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017)

BTCUSDT-1m já processado
BTCUSDT-5m já processado
BTCUSDT-15m já processado
BTCUSDT-30m já processado
BTCUSDT-1h já processado
BTCUSDT-2h já processado
BTCUSDT-4h já processado
BTCUSDT-8h já processado
BTCUSDT-1d já processado


In [ ]:
corrige_arquivos(timeframes)

## Cria janelas

In [5]:
# cria lista de qtd de minutos ou segundos por timeframe, dependendo de qual for o primeiro timeframe utilizado
# uso isso somente pra pegar a ultima linha que devemos iterar para criação de janelas
def timeframes_mesma_unidade(timeframes=timeframes):
    lista = []
    for timeframe in timeframes:
        qtd = int(timeframe[:-1])
        unidade = timeframe[-1]
        if timeframes[0] == "1s":
            lista.append(int(pd.to_timedelta(qtd, unit=unidade).total_seconds()))
        elif timeframes[0] == "1m":
            lista.append(int(pd.to_timedelta(qtd, unit=unidade).total_seconds()/60))
        else:
            print("Timeframe inicial não é de 1 minuto ou 1 segundo.")
    return(lista)


timeframes_padronizado = tuple(timeframes_mesma_unidade(timeframes))
timeframes_padronizado

(1, 5, 15, 30, 60, 120, 240, 480, 1440)

In [7]:
# cria uma lista de numpy arrays com os dados de cada timeframe.
# Eles estão organizados na mesma ordem que os timeframes
dfs = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1]).to_numpy(dtype="float32") for timeframe in timeframes]
dfs = tuple(dfs)
dfs[0]

array([[1.6542400e+04, 1.6725311e+12],
       [1.6536420e+04, 1.6725311e+12],
       [1.6537779e+04, 1.6725310e+12],
       ...,
       [4.2805601e+03, 1.5029426e+12],
       [4.2614800e+03, 1.5029425e+12],
       [4.2614800e+03, 1.5029425e+12]], dtype=float32)

In [8]:
# cria janela pra cada linha de dados
def cria_janela(linha, dfs=dfs , lookback=lookback):
    # cria janela vazia. precisa ter 1 coluna a mais que a qtd de timeframes
    df_janela = np.full(shape= (lookback, len(dfs)+1), fill_value=np.nan, dtype="float32")
    # coloca o preco de fechamento de 1m na primeira coluna e o close time de 1m na ultima coluna
    df_janela[:, 0] = dfs[0][linha:linha+lookback, 0]
    df_janela[:, -1] = dfs[0][linha:linha+lookback, -1]
    
    # define se a janela é long ou short
    # long se o preço de fechamento é maior no minuto seguinte, short se não, none se igual
    long = None
    if dfs[0][linha, 0] < dfs[0][linha-1, 0]:
        long = True
    elif dfs[0][linha, 0] > dfs[0][linha-1, 0]:
        long = False

    i = 1
    while i < len(dfs):
        # pega o numero da linha o primeiro closetime do timeframe é igual ou menor que o da janela atual
        index = (dfs[i][:,1] <= df_janela[0,-1]).argmax()
        # coloca as 20 (lookback) linhas subsequentes dentro da janela
        df_janela[:, i] = dfs[i][index:index+lookback, 0]
        i += 1
    # deleta coluna de close time e retorna
    return(df_janela[:,:-1], long)
#df_janela, long = cria_janela(99, dfs, lookback)
#print(long)
#pd.DataFrame(df_janela)

In [248]:
#%%timeit
# tentar meter um paralelismo. demora 3,4 min pra rodar atualmente

linha = 0

linhas = 10000
start = timer()
# ultima linha é a qtd de linhas de 1 min - (lookback (20 dias) x qtd de minutos em um dia) 
ultima_linha = len(dfs[0])-(lookback*timeframes_padronizado[-1])
print(ultima_linha)
for linha in range(linhas):# range(ultima_linha+1):
    df_janela, long = cria_janela(linha, dfs, lookback)
    # se não é long nem short cai fora
    if long == None:
        continue
    create_gaf(df_janela)
    # salva na pasta long ou short
end = timer()
print("Tempo de execução: ",end - start)
print("Tempo estimado para execução completa: ", (ultima_linha/linhas)*(end - start)/60, " min")
#pd.DataFrame(df_janela)

2789199
Tempo de execução:  6.527470999999423
Tempo estimado para execução completa:  30.344025976212315  min


# Problemas:
- Se for usar GAF image vai ter uma imagem 20x20 por coluna (supondo que estamos olhando 20 períodos no passado por vez)
- Não está claro se 1D CNN é uma boa alternativa para timeseries.

# Soluções:
#### 1
- Usar GAF de qualquer maneira para tentar replicar os resultados do trabalho do BARRA, usando apenas os preços de fechamento de 4 timeframes e ver se os resultados melhoram com CSVM
- Depois se eu quiser adiciono mais dados, mas a imagem vai ficar bem grande. (5 colunas de t-20 resulta em uma imagem de 100x100). GAF usando mais dados seria o melhor dos mundos.

#### 2
- Usar 1D CNN com todos os dados disponíveis
- Metodo completamente diferente do do BARRA. Não usa GAF images. Provavelmente mais fácil

### A partir daqui são só testes

In [249]:
# usando tambem markov
# ler o artigo https://arxiv.org/pdf/1506.00327.pdf pra entender melhor
def create_gaf(dft):
    # cria uma imagem gaf RGB
    # precisa da df_janela transposta pra funcionar
    gasf = GramianAngularField(method='summation')#, image_size=0.5)
    gadf = GramianAngularField(method='difference')#, image_size=0.5)
    mtf = MarkovTransitionField(n_bins=2)#, image_size=0.5)# , 
    gadf = gadf.transform(dft)
    gasf = gasf.transform(dft)
    mtf = mtf.transform(dft)
    #print(gadf.shape, gasf.shape, mtf.shape)
    
    # une os 3 metodos em uma unica imagem. precisa ter 3 dimensoes pra virar RGB, então criamos uma array vazia
    return np.stack((gasf,gadf, mtf), axis=-1)

In [251]:
import plotly.express as px

df_janela = cria_janela(99)[0]
img = create_gaf(df_janela.T)
print(img.shape[0], "imagens de dimensão", img.shape[1:3], "com", img.shape[3], "canais (RGB)")

# representação visual do que cada timeframe vira
fig = px.imshow(img[0, :, :, :], zmin=-1., zmax=1.)# color_continuous_scale='RdBu_r'
fig.show()

fig = px.imshow(img[0, :, :, 0], zmin=-1., zmax=1., color_continuous_scale='RdBu_r')
fig.show()

fig = px.imshow(img[0, :, :, 1], zmin=-1., zmax=1., color_continuous_scale='RdBu_r')
fig.show()

fig = px.imshow(img[0, :, :, 2], zmin=-1., zmax=1., color_continuous_scale='RdBu_r')
fig.show()

9 imagens de dimensão (20, 20) com 3 canais (RGB)


In [186]:
# tentar meter um paralelismo. demora 3,4 min pra rodar atualmente

linha = 0

linhas = 10000
start = timer()
# ultima linha é a qtd de linhas de 1 min - (lookback (20 dias) x qtd de minutos em um dia) 
ultima_linha = len(dfs[0])-(lookback*timeframes_padronizado[-1])
print(ultima_linha)
for linha in range(linhas):# range(ultima_linha+1):
    df_janela, long = cria_janela(linha, dfs, lookback)
    # se não é long nem short cai fora
    if long == None:
        continue
    # transforma em imagem
    create_gaf(df_janela.T)
    # salva na pasta long ou short
end = timer()
print("Tempo de execução: ",end - start)
print("Tempo estimado para execução completa: ", (ultima_linha/linhas)*(end - start)/60, " min")

2789199
Tempo de execução:  6.65773960000115
Tempo estimado para execução completa:  30.94960105763935  min


In [ ]:
# gera pra todos os tempos
# define se é long ou short
# transforma em gaf
# salva como imagem nas pastas long e short
# passa pro keras imagedatagenerator

## OLD

In [38]:
# cria um df com close times e close prices pra verificar as janelas
dfs = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[["Close", "Close time"]][::-1]).rename(columns={"Close" : f"Close_{timeframe}",
                                                                                                      "Close time" : f"Close time_{timeframe}"}).reset_index(drop=True) for timeframe in timeframes]

dfs_teste = pd.concat(dfs, axis=1)
for timeframe in timeframes:
    dfs_teste[f"Close time_{timeframe}"] = pd.to_datetime(dfs_teste[f"Close time_{timeframe}"], unit="ms")
dfs_teste[:2000].to_csv("teste.csv")

In [51]:
# cria um dicionário de dataframes. Pra acessar é só falar o timeframe desejado
dfs = {}
for timeframe in timeframes:
    dfs[timeframe] = (pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1])
dfs["1m"]

,Close,Close time
2817998,16542.40,1672531199999
2817997,16536.42,1672531139999
2817996,16537.78,1672531079999
2817995,16540.52,1672531019999
2817994,16539.28,1672530959999
...,...,...
4,4261.48,1502942699999
3,4261.48,1502942639999
2,4280.56,1502942579999
1,4261.48,1502942519999


In [52]:
%%time

# versao com query melhorada
# confirmei que funciona batendo o resultado com o da versao antiga e lenta
# ver se dá pra usar só numpy (fica mto escroto)
# e depois paralelismo

lookback = 20
linha = 0

colunas = [f"Close_{timeframe}" for timeframe in timeframes]
colunas.append("Close time")

# pra todas as linhas
print("Num linhas: ", len(dfs[timeframes[0]])-lookback)
#for linha in range(100):
for linha in [100]:
#for linha in range(len(df)-lookback):
    # cria o esqueleto da janela com o primeiro timeframe
    df_janela = pd.DataFrame(np.nan, index=range(0,lookback), columns= colunas)
    df_janela[[colunas[0], colunas[-1]]] = dfs[timeframes[0]].iloc[linha:linha+lookback].reset_index(drop=True)
    # junta o preço de fechamento pra todos os timeframes
    i = 1
    for timeframe in timeframes[1:]:
        # pega o numero da linha o primeiro closetime do timeframe é igual ou menor que o da janela atual
        index = (dfs[timeframe].iloc[:,1] <= df_janela.iat[0,-1]).values.argmax()
        # coloca as 20 (lookback) linhas subsequentes dentro da janela
        df_janela.iloc[:, i] = dfs[timeframe].iloc[index:index+lookback, 0].reset_index(drop=True)
        i += 1
    # remove a coluna close time, que foi utilizada pra sincronizar cada timeframe
    df_janela.drop("Close time", inplace=True, axis=1)
    # define se é long ou short
    # create_gaf(df_janela)
    # salva na pasta long ou short


df_janela2 = df_janela

Num linhas:  2817979
Wall time: 3.98 ms


In [31]:
# criação de DF gigante e cheio de nan's

def une_timeframes(timeframes=timeframes):
    # junta todos os timeframes em um df só

    # pega o primeiro timeframe com as colunas desejadas
    df = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframes[0]}.csv", index_col = "Close time"
                     , usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
    #df.index = pd.to_datetime(df.index, unit="ms")

    # coloca o sufixo nas colunas do menor timeframe
    df.columns = df.columns+f"_{timeframes[0]}"
    print(timeframes[0])

    # junta todos os outros timeframes e coloca o sufixo correto
    for timeframe in timeframes[1:]:
        print(timeframe)
        df2 = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", index_col = "Close time",
                         usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
        df2.columns = df2.columns+f"_{timeframe}"
        df = df.merge(df2, how="left", on=["Close time"])#, suffixes=(None, f"_{timeframe}"))


    # transforma o index em datetime
    df.index = pd.to_datetime(df.index, unit="ms")

    #inverte a ordem do df pq vamos usar ele do fim pro começo pra fazer as janelas
    df.sort_index(axis=0, inplace = True, ascending=False)
    return(df)
df = une_timeframes(timeframes)
df

1m
5m
15m
30m
1h
2h
4h
8h
1d


,Open_1m,High_1m,Low_1m,Close_1m,Volume_1m,Number of trades_1m,Open_5m,High_5m,Low_5m,Close_5m,...,Low_8h,Close_8h,Volume_8h,Number of trades_8h,Open_1d,High_1d,Low_1d,Close_1d,Volume_1d,Number of trades_1d
Close time,,,,,,,,,,,,,,,,,,,,,
2022-12-31 23:59:59.999,16536.81,16544.47,16536.23,16542.40,69.999440,2536,16540.42,16544.47,16535.05,16542.40,...,16470.0,16542.4,34408.36186,1011670.0,16607.48,16644.09,16470.0,16542.4,114490.42864,3344374.0
2022-12-31 23:58:59.999,16537.78,16538.81,16536.42,16536.42,28.882390,1267,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:57:59.999,16540.52,16540.79,16535.05,16537.78,53.359780,2088,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:56:59.999,16538.65,16540.53,16538.38,16540.52,38.588400,1536,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:55:59.999,16540.42,16540.92,16537.71,16539.28,36.236830,1551,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-17 04:04:59.999,4261.48,4261.48,4261.48,4261.48,0.140796,1,4261.48,4280.56,4261.48,4261.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 04:03:59.999,4261.48,4261.48,4261.48,4261.48,0.012008,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-17 04:02:59.999,4280.56,4280.56,4280.56,4280.56,0.261074,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
%%time
# solução de janela usando uma df gigante. Bem mais lento
# cria janela de precos
lookback = 20
linha = 0
# tentar fazer um loop pra pegar tudo

# pra todas as linhas
print("Num linhas: ", len(df)-lookback)
for linha in [2000000]:
#for linha in range(len(df)-lookback):
    # zera df_janela
    df_janela = pd.DataFrame()
    for timeframe in timeframes:
        # insere na ultima coluna; com o nome de Close_timeframe ;
        # df a partir da linha atual até linha atual + lookback
        #df_janela[f"Close_{timeframe}"] = df[f"Close_{timeframe}"].iloc[linha:].dropna().iloc[:lookback].reset_index(drop=True)
        df_janela.insert(len(df_janela.columns),(f"Close_{timeframe}"),df[f"Close_{timeframe}"].iloc[linha:].dropna().iloc[:lookback].to_list())
    #print(linha)
    # define se é long ou short
    # create_gaf(df_janela)
    # salva na pasta long ou short
df_janela
df_janela2 = df_janela
# funciona, mas está muito lento
# se tirar o dropna vai de 4,4 seg pra 190 ms
# não unir mais tudo

Num linhas:  2817979
Wall time: 12.7 ms


In [104]:
# sem rgb
from pyts.image import GramianAngularField
def create_gaf(df):
    """
    :param ts:
    :return:
    """
    #data = dict()
    gadf = GramianAngularField(method='difference')
    df = gadf.fit_transform(df.T) # ts.T)
    return df

In [112]:
pd.DataFrame(cria_janela(99)[0])

,0,1,2,3,4,5,6,7,8
0,16532.259766,16531.619141,16565.380859,16548.279297,16548.279297,16548.279297,16570.140625,16590.099609,16607.480469
1,16531.619141,16565.380859,16548.279297,16563.839844,16568.599609,16570.140625,16590.099609,16565.089844,16633.470703
2,16519.140625,16556.179688,16560.529297,16568.599609,16570.140625,16582.080078,16567.150391,16607.480469,16547.310547
3,16547.339844,16543.939453,16563.839844,16570.710938,16577.789062,16590.099609,16565.089844,16555.259766,16706.359375
4,16554.119141,16548.279297,16566.310547,16570.140625,16582.080078,16600.890625,16552.460938,16474.929688,16919.390625
5,16562.730469,16549.589844,16568.599609,16572.169922,16600.390625,16567.150391,16607.480469,16633.470703,16832.109375
6,16565.380859,16556.150391,16570.460938,16577.789062,16590.099609,16544.259766,16525.369141,16616.599609,16836.119141
7,16562.259766,16560.529297,16570.710938,16577.130859,16586.429688,16565.089844,16555.259766,16556.369141,16778.500000
8,16561.810547,16560.070312,16569.759766,16582.080078,16600.890625,16538.759766,16496.269531,16547.310547,16821.429688
9,16565.109375,16557.919922,16570.140625,16583.429688,16575.439453,16552.460938,16474.929688,16578.519531,16824.669922


In [153]:
def create_gaf(df):
    # cria uma imagem gaf RGB
    gasf = GramianAngularField(method='summation')
    gadf = GramianAngularField(method='difference')
    gadf = gadf.transform(df.T)
    gasf = gasf.transform(df.T)
    print(gadf.shape, gasf.shape)
    
    # une os 2 metodos em uma unica imagem. precisa ter 3 dimensoes pra virar RGB, então criamos uma array vazia
    df = np.stack((gasf,gadf, np.zeros(shape=gasf.shape)), axis=-1)
    return df

In [ ]:
# codigo pra tornar rgb
from pyts.image import GramianAngularField
import numpy as np

gasf = GramianAngularField(method='summation')
x_train_gasf = gasf.transform(x_train)
gadf = GramianAngularField(method='difference')
x_train_gadf = gadf.transform(x_train)
x_train_gaf=np.concatenate((x_train_gasf,x_train_gadf,np.zeros(x_train_gadf.shape)),axis=-1)

In [108]:
teste = pd.DataFrame(dfs[-1])#.to_excel("teste1.xlsx")
teste
#teste.loc[teste.iloc[:,0].isna() == True]

,0,1
0,17163.640625,1.669853e+12
1,16442.529297,1.669766e+12
2,16212.910156,1.669680e+12
3,16428.779297,1.669594e+12
4,16458.570312,1.669507e+12
...,...,...
1927,4016.000000,1.503360e+12
1928,4086.290039,1.503274e+12
1929,4139.979980,1.503187e+12
1930,4108.370117,1.503101e+12


In [111]:
df = pd.read_csv(f"./Dados/Processados/BTCUSDT-1m.csv", index_col = "Close time"
                     , usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
df.index = pd.to_datetime(df.index, unit="ms")
df

,Open,High,Low,Close,Volume,Number of trades
Close time,,,,,,
2017-08-17 04:00:59.999,4261.48,4261.48,4261.48,4261.48,1.775183,3
2017-08-17 04:01:59.999,4261.48,4261.48,4261.48,4261.48,0.000000,0
2017-08-17 04:02:59.999,4280.56,4280.56,4280.56,4280.56,0.261074,2
2017-08-17 04:03:59.999,4261.48,4261.48,4261.48,4261.48,0.012008,3
2017-08-17 04:04:59.999,4261.48,4261.48,4261.48,4261.48,0.140796,1
...,...,...,...,...,...,...
2022-11-30 23:55:59.999,17175.50,17215.59,17170.01,17173.33,531.365170,6445
2022-11-30 23:56:59.999,17173.33,17175.40,17150.70,17167.16,322.428780,7797
2022-11-30 23:57:59.999,17165.50,17170.80,17156.77,17161.26,160.506230,4056


# Falhas
Numba. Mais devagar mesmo 100% nopython. Talvez devido a falta de suporte ao argmax(), que fez com que eu tivesse que criar uma função pra encontrar o primeiro menor. No futuro, tentar pegar o indexador somando linhas.

In [7]:
# aparentemente o numba precisa de uma lista especializada quando é lista de listas
from numba.typed import List
import numba_funcs

dfs = numba_funcs.dfs_numba(timeframes)

In [8]:
lookback = 20

pd.DataFrame(numba_funcs.cria_janela(99, dfs, lookback))

,0,1,2,3,4,5,6,7,8
0,16532.259766,16531.619141,16565.380859,16548.279297,16548.279297,16548.279297,16570.140625,16590.099609,16607.480469
1,16531.619141,16565.380859,16548.279297,16563.839844,16568.599609,16570.140625,16590.099609,16565.089844,16633.470703
2,16519.140625,16556.179688,16560.529297,16568.599609,16570.140625,16582.080078,16567.150391,16607.480469,16547.310547
3,16547.339844,16543.939453,16563.839844,16570.710938,16577.789062,16590.099609,16565.089844,16555.259766,16706.359375
4,16554.119141,16548.279297,16566.310547,16570.140625,16582.080078,16600.890625,16552.460938,16474.929688,16919.390625
5,16562.730469,16549.589844,16568.599609,16572.169922,16600.390625,16567.150391,16607.480469,16633.470703,16832.109375
6,16565.380859,16556.150391,16570.460938,16577.789062,16590.099609,16544.259766,16525.369141,16616.599609,16836.119141
7,16562.259766,16560.529297,16570.710938,16577.130859,16586.429688,16565.089844,16555.259766,16556.369141,16778.500000
8,16561.810547,16560.070312,16569.759766,16582.080078,16600.890625,16538.759766,16496.269531,16547.310547,16821.429688
9,16565.109375,16557.919922,16570.140625,16583.429688,16575.439453,16552.460938,16474.929688,16578.519531,16824.669922


In [29]:
#ultima_linha = ((len(dfs[-1])-lookback-6)*timeframes_padronizado[-1])
# ultima linha é a qtd de linhas de 1 min - (lookback (20 dias) x qtd de minutos em um dia) 
ultima_linha = len(dfs[0])-(lookback*timeframes_padronizado[-1])
print(ultima_linha)
cria_janela(ultima_linha+200, dfs, lookback)

2789199


(array([[4395.  , 4395.  , 4415.07, 4415.07, 4366.47, 4366.47, 4366.47,
         4366.47, 4366.47],
        [4388.45, 4382.4 , 4368.01, 4366.47, 4423.46, 4374.43, 4442.48,
         4151.  , 4100.11],
        [4384.  , 4415.07, 4366.47, 4450.52, 4374.43, 4442.48, 4151.  ,
         4190.4 , 4509.08],
        [4383.85, 4403.  , 4426.27, 4423.46, 4431.61, 4322.47, 4246.82,
         4100.11, 4472.14],
        [4382.4 , 4367.99, 4450.52, 4413.99, 4442.48, 4151.  , 4190.4 ,
         4015.03, 4834.91],
        [4382.4 , 4368.01, 4425.  , 4374.43, 4390.14, 4261.04, 3988.04,
         4186.96, 4724.89],
        [4365.74, 4368.  , 4423.46, 4399.42, 4322.47, 4246.82, 4100.11,
         4509.08, 4555.14],
        [4365.74, 4368.05, 4415.72, 4431.61, 4142.69, 4120.05, 4203.67,
         4384.15, 4587.48],
        [4365.74, 4366.47, 4413.99, 4381.21, 4151.  , 4190.4 , 4015.03,
         4620.87, 4386.69],
        [4397.02, 4370.85, 4397.99, 4442.48, 4229.  , 3960.01, 4153.  ,
         4472.14, 4310.01],
